# SB1 Big Data Downloads
An analysis into the corridor analysis data downloads from Streetlight and Replica

In [117]:
import pandas as pd
from siuba import *

from calitp_data_analysis.sql import to_snakecase

import altair as alt
from calitp_data_analysis import calitp_color_palette as cp


In [2]:
pd.set_option('display.max_columns', 500)

In [3]:
GCS_PATH = "gs://calitp-analytics-data/data-analyses/sb125/corridor_study_data/"

### Read in Replica Data Spring 2023

In [4]:
replica_trips = "replica-bay_bridge-03_06_24-trips_dataset.csv"

In [5]:
r_trips = to_snakecase(pd.read_csv(f"{GCS_PATH}{replica_trips}"))

/tmp/ipykernel_339/2643890524.py:1: DtypeWarning: Columns (2,4,5,9,10,11,18,25,26,29,30,35,36,40,43,48,50) have mixed types. Specify dtype option on import or set low_memory=False.
  r_trips = to_snakecase(pd.read_csv(f"{GCS_PATH}{replica_trips}"))


In [6]:
r_trips.sample(5)

,origin_building_use,trip_duration_minutes,trip_taker_language,origin_trct_2020,trip_taker_household_id,trip_taker_sex,trip_taker_home_bgrp_2020,origin_st_2020,trip_taker_home_st_2020,trip_taker_race_ethnicity,transit_agency,transit_route,trip_taker_resident_type,vehicle_type,trip_taker_age,activity_id,destination_bgrp_2020,destination_building_use,trip_taker_wfh,destination_st_2020,destination_cty_2020,trip_taker_work_bgrp_2020,destination_trct_2020,trip_taker_household_income,origin_bgrp_2020,trip_taker_commute_mode,trip_taker_available_vehicles,primary_mode,previous_trip_purpose,trip_taker_building_type,transit_submode,trip_taker_work_st_2020,trip_taker_household_size,origin_cty_2020,destination_land_use,trip_taker_industry,trip_taker_tenure,vehicle_fuel_type,trip_taker_home_trct_2020,trip_taker_work_trct_2020,trip_taker_education,trip_start_time,trip_taker_individual_income,trip_taker_employment_status,trip_purpose,origin_land_use,trip_taker_work_cty_2020,trip_distance_miles,trip_taker_school_grade_attending,trip_taker_home_cty_2020,trip_taker_person_id,trip_end_time
267500,multi_family,53,english,"302.01 (San Francisco, CA)",2981999124390393386,male,"3 (Tract 302.01, San Francisco, CA)",California,California,two_races_not_hispanic_or_latino,NaN,NaN,core,unknown_vehicle_type,27.0,16597255654725327963,"1 (Tract 4222, Alameda, CA)",single_family,employed_not_working,California,Alameda,"3 (Tract 302.01, San Francisco, CA)","4222 (Alameda, CA)",284387.0,"3 (Tract 302.01, San Francisco, CA)",worked_from_home,three_plus,auto_passenger,home,single_family,NaN,California,3.0,San Francisco,single_family,not_working,renter,unknown_fuel_type,"302.01 (San Francisco, CA)","302.01 (San Francisco, CA)",bachelors_degree,08:04:00,94781.0,employed,school,multi_family,San Francisco,21.4,not_attending_school,San Francisco,114523536752271314,08:57:16
318811,education,51,GQ_language,"179.03 (San Francisco, CA)",294769040967747866,male,"1 (Tract 179.03, San Francisco, CA)",California,California,hispanic_or_latino_origin,NaN,NaN,core,unknown_vehicle_type,19.0,7487019993558144384,"2 (Tract 3131.04, Contra Costa, CA)",office,unemployed_under_16_not_in_labor_force,California,Contra Costa,Does not have work/school location,"3131.04 (Contra Costa, CA)",0.0,"1 (Tract 179.03, San Francisco, CA)",other_travel_mode,unknown_num_vehicles,private_auto,home,GQ_structure,NaN,Does not have work/school location,1.0,San Francisco,office,not_working,GQ,other_non_bev,"179.03 (San Francisco, CA)",Does not have work/school location,high_school,14:07:00,0.0,not_in_labor_force,maintenance,education,Does not have work/school location,35.2,not_attending_school,San Francisco,6449342572168120247,14:58:48
127266,multi_family,183,english,"102.01 (San Francisco, CA)",2007494479315805918,female,"2 (Tract 102.01, San Francisco, CA)",California,California,hispanic_or_latino_origin,NaN,NaN,core,unknown_vehicle_type,28.0,14874947884140784827,"2 (Tract 12.09, Nevada, CA)",retail,employed_not_working,California,Nevada,"1 (Tract 2014.01, Napa, CA)","12.09 (Nevada, CA)",327465.0,"2 (Tract 102.01, San Francisco, CA)",private_auto,three_plus,auto_passenger,home,multiple_units,NaN,California,3.0,San Francisco,retail,naics3399,renter,unknown_fuel_type,"102.01 (San Francisco, CA)","2014.01 (Napa, CA)",bachelors_degree,09:02:00,111762.0,employed,shop,multi_family,Napa,185.6,not_attending_school,San Francisco,15665869551117189445,12:05:47
309112,retail,28,spanish,"6022.02 (San Mateo, CA)",12449373272305106079,male,"2 (Tract 3810, Contra Costa, CA)",California,California,hispanic_or_latino_origin,NaN,NaN,core,unknown_vehicle_type,26.0,12179224938804524534,"2 (Tract 3810, Contra Costa, CA)",multi_family,unemployed_under_16_not_in_labor_force,California,Contra Costa,Does not have work/school location,"3810 (Contra Costa, CA)",0.0,"1 (Tract 6022.02, San Mateo, CA)",other_travel_mode,three_plus,auto_passenger,work,several_units,NaN,Does not have work/school location,2.0,San Mateo,multi_family,not_

In [148]:
r_trips.columns

Index(['origin_building_use', 'trip_duration_minutes', 'trip_taker_language',
       'origin_trct_2020', 'trip_taker_household_id', 'trip_taker_sex',
       'trip_taker_home_bgrp_2020', 'origin_st_2020',
       'trip_taker_home_st_2020', 'trip_taker_race_ethnicity',
       'transit_agency', 'transit_route', 'trip_taker_resident_type',
       'vehicle_type', 'trip_taker_age', 'activity_id',
       'destination_bgrp_2020', 'destination_building_use', 'trip_taker_wfh',
       'destination_st_2020', 'destination_cty_2020',
       'trip_taker_work_bgrp_2020', 'destination_trct_2020',
       'trip_taker_household_income', 'origin_bgrp_2020',
       'trip_taker_commute_mode', 'trip_taker_available_vehicles',
       'primary_mode', 'previous_trip_purpose', 'trip_taker_building_type',
       'transit_submode', 'trip_taker_work_st_2020',
       'trip_taker_household_size', 'origin_cty_2020', 'destination_land_use',
       'trip_taker_industry', 'trip_taker_tenure', 'vehicle_fuel_type',
       't

#### Data Exploration: Replica
* Summarizing the data that we exported from Replica
* Existing visualizations in replica: 
   * Primary Mode
   * Trip Purpose
   * Starting hour 
   * Trip Duration/Distance
   * Origin Destination
   * Vehicle Fuel type
   * Transit Routes/Stops/Sub mode/Agency
   * Household Income
   * Race and Ethnicity
   * Private Auto Availability
   * Age
   * Employment/School Status
   
   

In [138]:
trips = r_trips>>group_by(_.primary_mode)>>summarize(avg_trip_time = _.trip_duration_minutes.mean(),
                                                     number_trips = _.activity_id.nunique())

In [140]:
chart = (
        alt.Chart(trips)
        .mark_bar()
        .encode(
            x=alt.X("primary_mode"),
            y=alt.Y("avg_trip_time"),
            color=alt.Color("avg_trip_time", scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS,))
            )
      .properties(
    width=800,
    height=300)
        )
chart 

alt.Chart(...)

In [141]:
chart = (
        alt.Chart(trips)
        .mark_bar()
        .encode(
            x=alt.X("primary_mode"),
            y=alt.Y("number_trips"),
            color=alt.Color("number_trips", scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS,))
            )
      .properties(
    width=800,
    height=300)
        )
chart 

alt.Chart(...)

In [ ]:
len(r_trips)

674864

In [71]:
r_trips>>summarize(number_trips = _.activity_id.nunique())

,number_trips
0,674864


In [65]:
r_trips>>summarize(number_trips = _.trip_taker_person_id.nunique())

,number_trips
0,309596


In [97]:
race_ethnicity_by_mode = r_trips>>group_by(_.primary_mode)>>count(_.trip_taker_race_ethnicity)

In [116]:
chart = (
        alt.Chart(race_ethnicity_by_mode)
        .mark_bar()
        .encode(
            x=alt.X("primary_mode"),
            y=alt.Y("n"),
            color=alt.Color("trip_taker_race_ethnicity", scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS,))
            )
      .properties(
    width=800,
    height=300)
        )
chart 

alt.Chart(...)

In [144]:
# looking at transit mode, need to alter the transit_agency col and transit_submode to get a better count. 

(r_trips>>group_by(_.primary_mode, _.transit_submode)>>count(_.transit_agency)>>filter(_.transit_agency.notnull())>>arrange(-_.n)).head(15)

,primary_mode,transit_submode,transit_agency,n
4,public_transit,bus,AC TRANSIT,2788
16,public_transit,"bus, bus","San Francisco Municipal Transportation Agency,...",632
12,public_transit,"bus, bus","AC TRANSIT, San Francisco Municipal Transporta...",516
10,public_transit,"bus, bus","AC TRANSIT, AC TRANSIT",263
79,public_transit,"bus, light_rail","AC TRANSIT, San Francisco Municipal Transporta...",253
18,public_transit,"bus, bus","San Francisco Municipal Transportation Agency,...",234
7,public_transit,bus,San Francisco Municipal Transportation Agency,197
120,public_transit,"light_rail, bus, bus","San Francisco Municipal Transportation Agency,...",91
35,public_transit,"bus, bus, bus","San Francisco Municipal Transportation Agency,...",79
37,public_transit,"bus, bus, bus","San Francisco Municipal Transportation Agency,...",69


In [145]:
## most common transit mode combinations
(r_trips>>filter(_.primary_mode =="public_transit")>>count(_.transit_submode)>>arrange(-_.n)).head(15)

,transit_submode,n
0,bus,2999
1,"bus, bus",1706
21,"bus, light_rail",316
2,"bus, bus, bus",302
41,"light_rail, bus, bus",127
40,"light_rail, bus",76
55,"subway, bus",66
32,"bus, subway",63
14,"bus, bus, light_rail",49
33,"bus, subway, bus",46


In [8]:
r_trips>>count(_.trip_taker_household_id)>>arrange(-_.n)

,trip_taker_household_id,n
248686,,27897
262322,NaN,16384
110604,8194219563640751815,28
82299,6097771312917788596,27
74914,5552467881171814730,22
...,...,...
262312,9988489366890220188,1
262316,9991342309439330560,1
262317,9992490903429866665,1
262318,9992599661009943006,1


In [79]:
## checking one household id
r_trips>>filter(_.trip_taker_household_id == 8194219563640751815)

,origin_building_use,trip_duration_minutes,trip_taker_language,origin_trct_2020,trip_taker_household_id,trip_taker_sex,trip_taker_home_bgrp_2020,origin_st_2020,trip_taker_home_st_2020,trip_taker_race_ethnicity,transit_agency,transit_route,trip_taker_resident_type,vehicle_type,trip_taker_age,activity_id,destination_bgrp_2020,destination_building_use,trip_taker_wfh,destination_st_2020,destination_cty_2020,trip_taker_work_bgrp_2020,destination_trct_2020,trip_taker_household_income,origin_bgrp_2020,trip_taker_commute_mode,trip_taker_available_vehicles,primary_mode,previous_trip_purpose,trip_taker_building_type,transit_submode,trip_taker_work_st_2020,trip_taker_household_size,origin_cty_2020,destination_land_use,trip_taker_industry,trip_taker_tenure,vehicle_fuel_type,trip_taker_home_trct_2020,trip_taker_work_trct_2020,trip_taker_education,trip_start_time,trip_taker_individual_income,trip_taker_employment_status,trip_purpose,origin_land_use,trip_taker_work_cty_2020,trip_distance_miles,trip_taker_school_grade_attending,trip_taker_home_cty_2020,trip_taker_person_id,trip_end_time
115980,single_family,8,indo_european,"179.03 (San Francisco, CA)",8194219563640751815,male,"1 (Tract 179.03, San Francisco, CA)",California,California,hispanic_or_latino_origin,NaN,NaN,core,unknown_vehicle_type,28.0,11649742760210152290,"2 (Tract 180, San Francisco, CA)",industrial,in_person,California,San Francisco,"2 (Tract 180, San Francisco, CA)","180 (San Francisco, CA)",1200990.0,"1 (Tract 179.03, San Francisco, CA)",private_auto,three_plus,auto_passenger,home,single_family,NaN,California,10.0,San Francisco,industrial,naics31_33,renter,unknown_fuel_type,"179.03 (San Francisco, CA)","180 (San Francisco, CA)",bachelors_degree,07:26:00,136907.0,employed,work,single_family,San Francisco,4.3,not_attending_school,San Francisco,10066255028734967962,07:34:11
116099,industrial,8,indo_european,"180 (San Francisco, CA)",8194219563640751815,male,"1 (Tract 179.03, San Francisco, CA)",California,California,hispanic_or_latino_origin,NaN,NaN,core,unknown_vehicle_type,28.0,3527929229602445979,"1 (Tract 179.03, San Francisco, CA)",single_family,in_person,California,San Francisco,"2 (Tract 180, San Francisco, CA)","179.03 (San Francisco, CA)",1200990.0,"2 (Tract 180, San Francisco, CA)",private_auto,three_plus,auto_passenger,work,single_family,NaN,California,10.0,San Francisco,single_family,naics31_33,renter,unknown_fuel_type,"179.03 (San Francisco, CA)","180 (San Francisco, CA)",bachelors_degree,10:34:00,136907.0,employed,home,industrial,San Francisco,4.4,not_attending_school,San Francisco,10066255028734967962,10:42:11
257674,retail,20,indo_european,"4028.01 (Alameda, CA)",8194219563640751815,female,"1 (Tract 179.03, San Francisco, CA)",California,California,asian_not_hispanic_or_latino,NaN,NaN,core,unknown_vehicle_type,22.0,17607258221711465814,"1 (Tract 179.03, San Francisco, CA)",single_family,remote,California,San Francisco,"1 (Tract 179.03, San Francisco, CA)","179.03 (San Francisco, CA)",1200990.0,"1 (Tract 4028.01, Alameda, CA)",worked_from_home,three_plus,private_auto,shop,single_family,NaN,California,10.0,Alameda,single_family,naics62,renter,other_non_bev,"179.03 (San Francisco, CA)","179.03 (San Francisco, CA)",bachelors_degree,17:23:00,76878.0,employed,home,retail,San Francisco,8.3,not_attending_school,San Francisco,1325181746552868554,17:43:25
257680,retail,6,indo_european,"615.06 (San Francisco, CA)",8194219563640751815,male,"1 (Tract 179.03, San Francisco, CA)",California,California,white_not_hispanic_or_latino,NaN,NaN,core,unknown_vehicle_type,27.0,4258107892431763503,"1 (Tract 179.03, San Francisco, CA)",single_family,remote,California,San Francisco,"1 (Tract 179.03, San Francisco, CA)","179.03 (San Francisco, CA)",1200990.0,"3 (Tract 615.06, San Francisco, CA)",worked_from_home,three_plus,auto_passenger,shop,single_family,NaN,California,10.0,San Francisco,single_family,naics54,renter,unknown_fuel_type,"179.03 (San Francisco, CA)","179.03 (San Francisco, C

In [85]:
## checking one household id
r_trips>>filter(_.trip_taker_household_id == 8194219563640751815)>>count(_.trip_taker_person_id, _.trip_taker_age,
                                                                         _.trip_taker_sex, _.trip_taker_household_size)

,trip_taker_person_id,trip_taker_age,trip_taker_sex,trip_taker_household_size,n
0,1325181746552868554,22.0,female,10.0,6
1,2310331066900902679,30.0,female,10.0,6
2,4746831412975349070,34.0,female,10.0,2
3,10066255028734967962,28.0,male,10.0,4
4,11058477761487230854,32.0,male,10.0,2
5,12894946300265742193,39.0,male,10.0,2
6,14409692340574959811,27.0,male,10.0,6


### Read in Streetlight Data: 2022

In [10]:
streetlight = "streetlight_bay_bridge_corridor_study_corridor_study.csv"

In [11]:
sl_data = to_snakecase(pd.read_csv(f"{GCS_PATH}{streetlight}"))

In [12]:
sl_data.sample(5)

,data_periods,mode_of_travel,zone_id,zone_name,road_classification,line_zone_length__miles_,zone_is_pass_through,zone_direction__degrees_,zone_cardinal_direction,zone_is_bi_direction,day_type,day_part,average_daily_segment_traffic__stl_volume_,avg_segment_speed__mph_,avg_segment_travel_time__sec_,free_flow_speed__mph_,vehicle_miles_of_travel__stl_volume_,travel_time_index,congested_segment,_85th_speed_percentile
1249,"Jan 01, 2022 - Dec 31, 2022",All Vehicles CVD Plus - StL All Vehicles Volume,1125811810,I 80 / 839972874 / 1,Motorway,0.111,yes,85,EAST,no,2: Weekend Day (Sa-Su),19: 3pm (3pm-4pm),7876,62,7,70.037,875.09,1.13,False,70
1024,"Jan 01, 2022 - Dec 31, 2022",All Vehicles CVD Plus - StL All Vehicles Volume,1118036302,I 80 / 839972875 / 1,Motorway,0.006,yes,265,WEST,no,1: Weekday (M-F),04: 2am (2am-3am),682,67,2,68.762,3.79,1.02,False,75
559,"Jan 01, 2022 - Dec 31, 2022",All Vehicles CVD Plus - StL All Vehicles Volume,1093181568,San Francisco – Oakland Bay Bridge / 50691047 / 1,Motorway,0.109,yes,40,EAST,no,0: All Days (M-Su),19: 3pm (3pm-4pm),9364,40,9,66.047,1020.38,1.64,True,56
722,"Jan 01, 2022 - Dec 31, 2022",All Vehicles CVD Plus - StL All Vehicles Volume,1112062603,San Francisco – Oakland Bay Bridge / 236348361...,Motorway,1.878,yes,260,WEST,no,0: All Days (M-Su),02: Early AM (12am-6am),15876,64,107,70.409,29817.20,1.09,False,74
1317,"Jan 01, 2022 - Dec 31, 2022",All Vehicles CVD Plus - StL All Vehicles Volume,1128174334,San Francisco – Oakland Bay Bridge / 237731428...,Motorway,0.177,yes,43,EAST,no,1: Weekday (M-F),27: 9pm (9pm-10pm),6538,60,11,67.749,1156.19,1.13,False,67


#### Data Explorations: Streetlight

In [13]:
len(sl_data)

1710

In [14]:
sl_data>>count(_.zone_id, _.zone_name)

,zone_id,zone_name,n
0,1005898592,San Francisco – Oakland Bay Bridge / 8921938 / 1,90
1,1020456908,San Francisco – Oakland Bay Bridge / 23874736 / 1,90
2,1022959400,San Francisco – Oakland Bay Bridge / 661905446...,90
3,1032819756,San Francisco – Oakland Bay Bridge / 11415208 / 1,90
4,1036056766,San Francisco – Oakland Bay Bridge / 52721870 / 1,90
5,1050419851,San Francisco – Oakland Bay Bridge / 179235221...,90
6,1093181568,San Francisco – Oakland Bay Bridge / 50691047 / 1,90
7,1112052563,San Francisco – Oakland Bay Bridge / 236348360...,90
8,1112062603,San Francisco – Oakland Bay Bridge / 236348361...,90
9,1114495634,I 80 / 395419317 / 1,90


In [15]:
sl_data>>filter(_.zone_id==1133975975)>>count(_.day_part)

,day_part,n
0,00: All Day (12am-12am),3
1,01: 12am (12am-1am),3
2,02: Early AM (12am-6am),3
3,03: 1am (1am-2am),3
4,04: 2am (2am-3am),3
5,05: 3am (3am-4am),3
6,06: 4am (4am-5am),3
7,07: 5am (5am-6am),3
8,08: 6am (6am-7am),3
9,09: Peak AM (6am-10am),3


* The data here shows that each segment is broken out into hour time slots and then also aggregated to peak time periods. moving forward we will look at the `All Day` for `day_part` and `All Days` for `day_type`

In [16]:
sl_data_single_day = sl_data>>filter(_.day_part == '00: All Day (12am-12am)', _.day_type == '0: All Days (M-Su)')

In [17]:
len(sl_data_single_day)

19

In [18]:
sl_data_single_day>>count(_.zone_id, _.zone_name)

,zone_id,zone_name,n
0,1005898592,San Francisco – Oakland Bay Bridge / 8921938 / 1,1
1,1020456908,San Francisco – Oakland Bay Bridge / 23874736 / 1,1
2,1022959400,San Francisco – Oakland Bay Bridge / 661905446...,1
3,1032819756,San Francisco – Oakland Bay Bridge / 11415208 / 1,1
4,1036056766,San Francisco – Oakland Bay Bridge / 52721870 / 1,1
5,1050419851,San Francisco – Oakland Bay Bridge / 179235221...,1
6,1093181568,San Francisco – Oakland Bay Bridge / 50691047 / 1,1
7,1112052563,San Francisco – Oakland Bay Bridge / 236348360...,1
8,1112062603,San Francisco – Oakland Bay Bridge / 236348361...,1
9,1114495634,I 80 / 395419317 / 1,1


In [19]:
sl_data_single_day>>filter(_.zone_id==1133975975)

,data_periods,mode_of_travel,zone_id,zone_name,road_classification,line_zone_length__miles_,zone_is_pass_through,zone_direction__degrees_,zone_cardinal_direction,zone_is_bi_direction,day_type,day_part,average_daily_segment_traffic__stl_volume_,avg_segment_speed__mph_,avg_segment_travel_time__sec_,free_flow_speed__mph_,vehicle_miles_of_travel__stl_volume_,travel_time_index,congested_segment,_85th_speed_percentile
1530,"Jan 01, 2022 - Dec 31, 2022",All Vehicles CVD Plus - StL All Vehicles Volume,1133975975,San Francisco – Oakland Bay Bridge / 929579737...,Motorway,0.273,yes,215,WEST,no,0: All Days (M-Su),00: All Day (12am-12am),128139,49,23,68.634,35038.3,1.39,True,67


In [22]:
sl_data_single_day.sample(3)

,data_periods,mode_of_travel,zone_id,zone_name,road_classification,line_zone_length__miles_,zone_is_pass_through,zone_direction__degrees_,zone_cardinal_direction,zone_is_bi_direction,day_type,day_part,average_daily_segment_traffic__stl_volume_,avg_segment_speed__mph_,avg_segment_travel_time__sec_,free_flow_speed__mph_,vehicle_miles_of_travel__stl_volume_,travel_time_index,congested_segment,_85th_speed_percentile
450,"Jan 01, 2022 - Dec 31, 2022",All Vehicles CVD Plus - StL All Vehicles Volume,1050419851,San Francisco – Oakland Bay Bridge / 179235221...,Motorway,0.004,yes,38,EAST,no,0: All Days (M-Su),00: All Day (12am-12am),161170,53,2,67.840,695.17,1.29,True,67
540,"Jan 01, 2022 - Dec 31, 2022",All Vehicles CVD Plus - StL All Vehicles Volume,1093181568,San Francisco – Oakland Bay Bridge / 50691047 / 1,Motorway,0.109,yes,40,EAST,no,0: All Days (M-Su),00: All Day (12am-12am),133795,51,8,66.047,14579.50,1.30,True,64
0,"Jan 01, 2022 - Dec 31, 2022",All Vehicles CVD Plus - StL All Vehicles Volume,1005898592,San Francisco – Oakland Bay Bridge / 8921938 / 1,Motorway,1.985,yes,220,WEST,no,0: All Days (M-Su),00: All Day (12am-12am),108456,44,179,66.336,215328.00,1.50,True,63


In [29]:
sl_data_single_day>>group_by(_.zone_cardinal_direction, _.day_type, _.day_part)>>summarize(avg_speed= _.avg_segment_speed__mph_.mean(),
                                                                                          avg_travel_time = _.avg_segment_travel_time__sec_.mean(),
                                                                                          avg_volume = _.average_daily_segment_traffic__stl_volume_.mean(),
                                                                                          sum_volume = _.average_daily_segment_traffic__stl_volume_.sum())

,zone_cardinal_direction,day_type,day_part,avg_speed,avg_travel_time,avg_volume,sum_volume
0,EAST,0: All Days (M-Su),00: All Day (12am-12am),55.900000,32.600000,139682.300000,1396823
1,WEST,0: All Days (M-Su),00: All Day (12am-12am),50.333333,41.777778,123029.777778,1107268


In [38]:
sl_peaks = (sl_data>>filter(
                            _.day_part != "01: 12am (12am-1am)",
                            _.day_part != "03: 1am (1am-2am)",
                            _.day_part != "04: 2am (2am-3am)",
                            _.day_part != "05: 3am (3am-4am)",
                            _.day_part != "06: 4am (4am-5am)",
                            _.day_part != "07: 5am (5am-6am)",
                            _.day_part != "08: 6am (6am-7am)",
                            _.day_part != "10: 7am (7am-8am)",
                            _.day_part != "11: 8am (8am-9am)",
                            _.day_part != "12: 9am (9am-10am)",
                            _.day_part != "13: 10am (10am-11am)",
                            _.day_part != "15: 11am (11am-12noon)",
                            _.day_part != "16: 12pm (12noon-1pm)",
                            _.day_part != "17: 1pm (1pm-2pm)",
                            _.day_part != "18: 2pm (2pm-3pm)",
                            _.day_part != "19: 3pm (3pm-4pm)",
                            _.day_part != "20: 4pm (4pm-5pm)",
                            _.day_part != "22: 5pm (5pm-6pm)",
                            _.day_part != "23: 6pm (6pm-7pm)",
                            _.day_part != "24: 7pm (7pm-8pm)",
                            _.day_part != "25: 8pm (8pm-9pm)",
                            _.day_part != "27: 9pm (9pm-10pm)",
                            _.day_part != "28: 10pm (10pm-11pm)",
                            _.day_part !="29: 11pm (11pm-12am)"))

In [35]:
sl_hourly = (sl_data>>filter(_.day_part != "00: All Day (12am-12am)",
                            _.day_part != "02: Early AM (12am-6am)",
                            _.day_part != "09: Peak AM (6am-10am)",
                            _.day_part != "14: Mid-Day (10am-4pm)",
                            _.day_part != "21: Peak PM (4pm-8pm)", _.day_part !="26: Late PM (8pm-12am)"))

In [49]:
sl_peaks_agg = sl_peaks>>group_by(_.zone_cardinal_direction,
    _.day_type, _.day_part)>>summarize(avg_speed= _.avg_segment_speed__mph_.mean(),
                                                                                          avg_travel_time = _.avg_segment_travel_time__sec_.mean(),
                                                                                          avg_volume = _.average_daily_segment_traffic__stl_volume_.mean(),
                                                                                          sum_volume = _.average_daily_segment_traffic__stl_volume_.sum())

In [154]:
sl_peaks_agg.sample()

,zone_cardinal_direction,day_type,day_part,avg_speed,avg_travel_time,avg_volume,sum_volume
34,WEST,2: Weekend Day (Sa-Su),21: Peak PM (4pm-8pm),40.222222,54.222222,23846.222222,214616


In [157]:
chart = (
        alt.Chart(sl_peaks_agg)
        .mark_bar()
        .encode(
            x=alt.X("day_part"),
            y=alt.Y("avg_speed"),
            color=alt.Color("zone_cardinal_direction", scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS,
            domain=sl_peaks_agg["zone_cardinal_direction"].unique().tolist())
            ),
            tooltip=sl_peaks_agg.columns.tolist(),
            )
      .properties(
    width=800,
    height=300)
        )
chart 

alt.Chart(...)

In [155]:
chart = (
        alt.Chart(sl_peaks_agg)
        .mark_bar()
        .encode(
            x=alt.X("day_part"),
            y=alt.Y("avg_speed"),
            color=alt.Color("zone_cardinal_direction", scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS,
            domain=sl_peaks_agg["zone_cardinal_direction"].unique().tolist())
            ),
            tooltip=sl_peaks_agg.columns.tolist(),
            )
      .properties(
    width=800,
    height=300)
        )
chart 

alt.Chart(...)

In [48]:
sl_hourly_agg = sl_hourly>>filter(_.day_type=="0: All Days (M-Su)")>>group_by(_.zone_cardinal_direction,
    _.day_type, _.day_part)>>summarize(avg_speed= _.avg_segment_speed__mph_.mean(),
                                                                                          avg_travel_time = _.avg_segment_travel_time__sec_.mean(),
                                                                                          avg_volume = _.average_daily_segment_traffic__stl_volume_.mean(),
                                                                                          sum_volume = _.average_daily_segment_traffic__stl_volume_.sum())

In [153]:
sl_hourly_agg.sample()

,zone_cardinal_direction,day_type,day_part,avg_speed,avg_travel_time,avg_volume,sum_volume
3,EAST,0: All Days (M-Su),05: 3am (3am-4am),66.7,25.6,741.0,7410


In [151]:
chart = (
        alt.Chart(sl_hourly_agg)
        .mark_bar()
        .encode(
            x=alt.X("day_part"),
            y=alt.Y("avg_speed"),
            color=alt.Color("zone_cardinal_direction", scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS,
            domain=sl_hourly_agg["zone_cardinal_direction"].unique().tolist())
            ),
            tooltip=sl_hourly_agg.columns.tolist(),
            )
      .properties(
    width=800,
    height=300)
        )
chart 

alt.Chart(...)

In [152]:
chart = (
        alt.Chart(sl_hourly_agg)
        .mark_bar()
        .encode(
            x=alt.X("day_part"),
            y=alt.Y("sum_volume"),
            color=alt.Color("zone_cardinal_direction", scale=alt.Scale(range = cp.CALITP_DIVERGING_COLORS,
            domain=sl_hourly_agg["zone_cardinal_direction"].unique().tolist())
            ),
            tooltip=sl_hourly_agg.columns.tolist(),
            )
      .properties(
    width=800,
    height=300)
        )
chart 

alt.Chart(...)